In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import joblib
import time

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import mne
from mne import Epochs, pick_types, annotations_from_events, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP, SPoC
from mne.viz import plot_events, plot_montage
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs, corrmap, Xdawn

mne.set_log_level("CRITICAL")

In [2]:
tmin, tmax = -1.0, 4.0
subjects_count = 109

In [3]:
experiments = [
    {
        "runs": [3, 7, 11],
        "mapping": {0: "rest", 1: "left fist", 2: "right fist"},
    },
    {
        "runs": [4, 8, 12],
        "mapping": {0: "rest", 1: "imagine left fist", 2: "imagine right fist"},
    },
    {
        "runs": [5, 9, 13],
        "mapping": {0: "rest", 1: "both fists", 2: "both feets"},
    },
    {
        "runs": [6, 10, 14],
        "mapping": {0: "rest", 1: "imagine both fists", 2: "imagine both feets"},
    },
    # ----------------------------------------------------------------------------
    # {
    #     "runs": [3, 7, 11, 4, 8, 12],
    #     "mapping": {0: "rest", 1: "left fist", 2: "right fist"},
    # },
    # {
    #     "runs": [5, 9, 13, 6, 10, 14],
    #     "mapping": {0: "rest", 1: "both fists", 2: "both feets"},
    # },
]

In [4]:
# Create initial models
models = []
for i in range(4):
    models.append(joblib.load(f"models/model_{i}.z"))

In [7]:
all_accuracies = []
for subject in range(1, subjects_count + 1):
    print(f"Subject #{subject}")

    for model_id, experiment in enumerate(experiments):
        raw_fnames = [f"dataset/S{subject:03d}/S{subject:03d}R{run:02d}.edf" for run in experiment["runs"]]
        raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
        events, _ = events_from_annotations(raw, event_id=dict(T1=1, T2=2))
        annot_from_events = annotations_from_events(
            events=events, event_desc=experiment["mapping"], sfreq=raw.info["sfreq"]
        )
        raw.set_annotations(annot_from_events)
        eegbci.standardize(raw)  # set channel names
        montage = make_standard_montage("biosemi64")
        raw.set_montage(montage, on_missing='ignore')

        # Select channels
        channels = raw.info["ch_names"] 
        good_channels = [
            "FC3",
            "FC1",
            "FCz",
            "FC2",
            "FC4",
            "C3",
            "C1",
            "Cz",
            "C2",
            "C4",
            "CP3",
            "CP1",
            "CPz",
            "CP2",
            "CP4",
            "Fpz",
        ]
        bad_channels = [x for x in channels if x not in good_channels]
        raw.drop_channels(bad_channels)

        # Filter
        raw.notch_filter(60, method="iir")
        raw.filter(7, 32.0, fir_design="firwin", skip_by_annotation="edge") 

        # Read epochs
        events, event_id = events_from_annotations(raw)
        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")
        epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
        epochs_data = epochs.get_data()
        labels = epochs.events[:, -1]

        # Score
        start = time()
        accuracy = accuracy_score(labels, models[model_id].predict(epochs_data))
        print(f"[Training] Accuracy: {accuracy:.2%}")
        all_accuracies.append(accuracy)

print(f"Accuracy: {np.mean(all_accuracies):.2%}")

Subject #1
[Training] Accuracy: 64.44%
[Training] Accuracy: 60.00%
[Training] Accuracy: 53.33%
[Training] Accuracy: 51.11%
Subject #2
[Training] Accuracy: 51.11%
[Training] Accuracy: 51.11%
[Training] Accuracy: 44.44%
[Training] Accuracy: 46.67%
Subject #3
[Training] Accuracy: 48.89%
[Training] Accuracy: 55.56%
[Training] Accuracy: 48.89%
[Training] Accuracy: 46.67%
Subject #4
[Training] Accuracy: 66.67%
[Training] Accuracy: 48.89%
[Training] Accuracy: 48.89%
[Training] Accuracy: 55.56%
Subject #5
[Training] Accuracy: 57.78%
[Training] Accuracy: 53.33%
[Training] Accuracy: 48.89%
[Training] Accuracy: 51.11%
Subject #6
[Training] Accuracy: 48.89%
[Training] Accuracy: 44.44%
[Training] Accuracy: 46.67%
[Training] Accuracy: 51.11%
Subject #7
[Training] Accuracy: 53.33%
[Training] Accuracy: 53.33%
[Training] Accuracy: 48.89%
[Training] Accuracy: 62.22%
Subject #8
[Training] Accuracy: 46.67%
[Training] Accuracy: 55.56%
[Training] Accuracy: 48.89%
[Training] Accuracy: 48.89%
Subject #9
[Trai